In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import VarianceThreshold
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import VarianceThreshold
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Path to your CSV file
file_path = 'diabetes.csv'

# Read the CSV file into a DataFrame
pima = pd.read_csv(file_path)

# Display the contents of the DataFrame
print(pima)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   50        1  
1                  

In [3]:
# Display features
print("Features:\n", pima.columns)

Features:
 Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')


In [4]:
pip install imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Data Sampling

In [5]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# Display the class distribution before SMOTE
print("Class distribution before SMOTE:", Counter(pima['Outcome']))

# Separate the features and the target variable
X = pima.drop('Outcome', axis=1)
y = pima['Outcome']

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

# Display the class distribution after SMOTE
print("Class distribution after SMOTE:", Counter(y_smote))

Class distribution before SMOTE: Counter({0: 500, 1: 268})
Class distribution after SMOTE: Counter({1: 500, 0: 500})


## Missing Values

In [6]:
# Check for missing values in the dataset
missing_values = pima.isnull().sum()
print("Missing values in each column:\n", missing_values)

Missing values in each column:
 Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [7]:
# Replace zero values with NaN for the specified columns
columns_to_replace = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
X_smote[columns_to_replace] = X_smote[columns_to_replace].replace(0, np.nan)

# Check for missing values again in the resampled dataset after replacing 0 with NaN
missing_values_resampled_updated = X_smote.isnull().sum()
print("\nMissing values in each column after replacing 0 with NaN:")
print(missing_values_resampled_updated)

# Display a summary of the total missing values in the updated resampled dataset
total_missing_resampled_updated = missing_values_resampled_updated.sum()
print(f"\nTotal missing values in the updated resampled dataset: {total_missing_resampled_updated}")

# Optionally, display columns with missing values only in the updated resampled dataset
missing_values_only_resampled_updated = missing_values_resampled_updated[missing_values_resampled_updated > 0]
if not missing_values_only_resampled_updated.empty:
    print("\nColumns with missing values in the updated resampled dataset:")
    print(missing_values_only_resampled_updated)
else:
    print("\nNo missing values in the updated resampled dataset.")

# Calculate the mean of each column
mean_values = X_smote.mean()

# Replace NaN values with the mean of each column
X_smote.fillna(mean_values, inplace=True)

# Check again for missing values in the dataset
missing_values_after = X_smote.isnull().sum()
print("Missing values in each column after replacement:\n", missing_values_after)


Missing values in each column after replacing 0 with NaN:
Pregnancies                   0
Glucose                       5
BloodPressure                51
SkinThickness               295
Insulin                     488
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
dtype: int64

Total missing values in the updated resampled dataset: 850

Columns with missing values in the updated resampled dataset:
Glucose            5
BloodPressure     51
SkinThickness    295
Insulin          488
BMI               11
dtype: int64
Missing values in each column after replacement:
 Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64


In [8]:
# Print the current dataset after data cleaning
print("\nDataset after data cleaning:")
print(X_smote)


Dataset after data cleaning:
     Pregnancies  Glucose  BloodPressure  SkinThickness     Insulin  \
0              6    148.0           72.0      35.000000  165.414062   
1              1     85.0           66.0      29.000000  165.414062   
2              8    183.0           64.0      29.680851  165.414062   
3              1     89.0           66.0      23.000000   94.000000   
4              0    137.0           40.0      35.000000  168.000000   
..           ...      ...            ...            ...         ...   
995            5    164.0           64.0      29.680851  165.414062   
996            5    107.0           69.0      31.000000  165.414062   
997            4    171.0           83.0      27.000000  154.000000   
998            8    111.0           81.0      32.000000  175.000000   
999            4    144.0           79.0      32.000000  165.414062   

           BMI  DiabetesPedigreeFunction  Age  
0    33.600000                  0.627000   50  
1    26.600000       

## Feature Selection

In [11]:
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Feature selection using RFE
svc = SVC(kernel='linear')
rfe = RFE(estimator=svc, n_features_to_select=6, step=1)
rfe.fit(X_smote, y_smote)

# Print the features that RFE chooses
selected_features = X_smote.columns[rfe.support_]
print("Selected features by RFE:\n", selected_features)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_smote[selected_features], y_smote, test_size=0.2, random_state=42)

# Create and train an SVM model
svm_model = SVC(kernel='linear', C=1)
svm_model.fit(X_train, y_train)

# Predict using the SVM model
y_pred_svm = svm_model.predict(X_test)

# Calculate and print the performance metrics for SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)

print("\nSVM Model Performance:")
print(f"Accuracy: {accuracy_svm:.2f}")
print(f"Recall: {recall_svm:.2f}")
print(f"Precision: {precision_svm:.2f}")
print(f"F1-Score: {f1_svm:.2f}")

# Create and train a Decision Tree model
dt_model = DecisionTreeClassifier(criterion='gini')
dt_model.fit(X_train, y_train)

# Predict using the Decision Tree model
y_pred_dt = dt_model.predict(X_test)

# Calculate and print the performance metrics for Decision Tree
accuracy_dt = accuracy_score(y_test, y_pred_dt)
recall_dt = recall_score(y_test, y_pred_dt)
precision_dt = precision_score(y_test, y_pred_dt)
f1_dt = f1_score(y_test, y_pred_dt)

print("\nDecision Tree Model Performance:")
print(f"Accuracy: {accuracy_dt:.2f}")
print(f"Recall: {recall_dt:.2f}")
print(f"Precision: {precision_dt:.2f}")
print(f"F1-Score: {f1_dt:.2f}")


Selected features by RFE:
 Index(['Pregnancies', 'Glucose', 'SkinThickness', 'BMI',
       'DiabetesPedigreeFunction', 'Age'],
      dtype='object')

SVM Model Performance:
Accuracy: 0.76
Recall: 0.78
Precision: 0.75
F1-Score: 0.76

Decision Tree Model Performance:
Accuracy: 0.73
Recall: 0.74
Precision: 0.74
F1-Score: 0.74
